In [18]:
import pandas as pd
from diive.core.io.files import save_parquet, load_parquet

# Read flux data

In [19]:
FILEPATH = r"../50_FluxProcessingChain/53.1_FluxProcessingChain_after-L3.3_NEE_LE_H.parquet"
maindf = load_parquet(filepath=FILEPATH)

Loaded .parquet file ../50_FluxProcessingChain/53.1_FluxProcessingChain_after-L3.3_NEE_LE_H.parquet (4.108 seconds). Detected time resolution of <30 * Minutes> / 30min 


# Read management data

In [142]:
file_mgmt = "CH-CHA-Management_2005-2024_20240918.xlsx"
sheet = "CHA_Management"
df = pd.read_excel(file_mgmt, sheet_name=sheet)
df = df.loc[:, ['Parcel', 'Management_Category', 'Management_Subcategory', 'Management_specific', 'Start', 'End']]

df['PREFIX'] = 'MGMT'

# Fill end dates, if missing then end is the same as start
df['End'] = df['End'].fillna(df['Start'])
df['Parcel'] = df['Parcel'].fillna('-9999')
df['Management_Category'] = df['Management_Category'].fillna('-9999')
df['Management_Subcategory'] = df['Management_Subcategory'].fillna('-9999')
df['Management_specific'] = df['Management_specific'].fillna('-9999')

df['Parcel'] = df['Parcel'].str.replace('A', 'PARCEL-A')
df['Parcel'] = df['Parcel'].str.replace('B', 'PARCEL-B')
df['Management_Category'] = df['Management_Category'].str.replace('Fertilization', 'FERT')
df['Management_Category'] = df['Management_Category'].str.replace('Plant protection and growth maintenance', 'PPGM')
df['Management_Category'] = df['Management_Category'].str.replace('Use', 'USE')
df['Management_Category'] = df['Management_Category'].str.replace('Sowing', 'SOWING')
df['Management_Category'] = df['Management_Category'].str.replace('Soil cultivation', 'SOILCULTIVATION')

df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Organic fertilizer', 'ORG')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Mineral fertilizer', 'MIN')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Herbe removal', 'HERBREMOVAL')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Harrowing', 'HARROWING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Grazing', 'GRAZING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Rolling', 'ROLLING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Oversowing', 'OVERSOWING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Mowing', 'MOWING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Herbicide', 'HERBICIDE')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Ploughing', 'PLOUGHING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Resowing', 'RESOWING')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Pesticide', 'PESTICIDE')
df['Management_Subcategory'] = df['Management_Subcategory'].str.replace('Tillage', 'TILLAGE')

df['Management_specific'] = df['Management_specific'].str.replace('Manure', 'MAN')
df['Management_specific'] = df['Management_specific'].str.replace('Slurry', 'SLU')
df['Management_specific'] = df['Management_specific'].str.replace('Calcium ammonium nitrate', 'CAN')
df['Management_specific'] = df['Management_specific'].str.replace('Molluscicide (applied on the whole field)', 'MOLLUSCICIDE')
df['Management_specific'] = df['Management_specific'].str.replace('Harmony tabs (applied on single plants against Rumex obtusifolius)', 'HARMONYTABS')
df['Management_specific'] = df['Management_specific'].str.replace('Ally Tabs (applied on single plants against Rumex spp.)', 'ALLYTABS')
df['Management_specific'] = df['Management_specific'].str.replace('Ally Tabs (applied against Rumex spp. and Senecio spp.)', 'ALLYTABS')
df['Management_specific'] = df['Management_specific'].str.replace('Hay', 'HAY')
df['Management_specific'] = df['Management_specific'].str.replace('Sheep', 'SHEEP')
df['Management_specific'] = df['Management_specific'].str.replace('Cattle', 'CATTLE')
df['Management_specific'] = df['Management_specific'].str.replace('Aftergrass', 'AFTERGRASS')
df['Management_specific'] = df['Management_specific'].str.replace('Asulox (applied on the whole field against Rumex obtusifolius)', 'ASULOX')
df['Management_specific'] = df['Management_specific'].str.replace('Rumex removal', 'RUMEX')
df['Management_specific'] = df['Management_specific'].str.replace('Harrowing (1 cm)', '1CM')
df['Management_specific'] = df['Management_specific'].str.replace('Harrowing (7 cm)', '7CM')
df['Management_specific'] = df['Management_specific'].str.replace('Roundup PowerMax', 'ROUNDUP')

df['Management_specific'] = df['Management_specific'].str.replace('Type OH 440 Extra', 'OH440EXTRA')
df['Management_specific'] = df['Management_specific'].str.replace('?', '-9999')
df['Management_specific'] = df['Management_specific'].str.replace('Type OH 440 RenoTurbo', 'OH440RENOTURBO')
df['Management_specific'] = df['Management_specific'].str.replace('Type OH 440 Reno', 'OH440RENO')
df['Management_specific'] = df['Management_specific'].str.replace('Type OH 240 Reno', 'OH240RENO')

df['Management_specific'] = df['Management_specific'].str.replace('Asulam (applied on the whole field against Rumex obtusifolius)', 'ASULAM')
# df['Management_specific'] = df['Management_specific'].str.replace('HEBE FIONA MERULA. Sorte HEBE: Trifolium repens (small leaves). Sorte FIONA: Trifolium repens (big leaves). Sorte MARULA: Trifolium pratense. Total 20 kg/ha. Mixture: 27% HEBE  36% FIONA 36% MERULA (15 kg + 20 kg + 20 kg)', '-9999')




df['Management_specific'] = df['Management_specific'].str.replace('Grass silage', 'GRASSSILAGE')
df['Management_specific'] = df['Management_specific'].str.replace('UFA 420', 'UFA-420')

s = "HEBE FIONA MERULA. Sorte HEBE: Trifolium repens (small leaves). Sorte FIONA: Trifolium repens (big leaves). Sorte MERULA: Trifolium pratense. Total 20 kg/ha. Mixture: 1/4 HEBE  1/2 FIONA 1/4 MERULA"
df['Management_specific'] = df['Management_specific'].str.replace(s, 'HEBE-FIONA-MERULA')

s = "HEBE FIONA MERULA. Sorte HEBE: Trifolium repens (small leaves). Sorte FIONA: Trifolium repens (big leaves). Sorte MARULA: Trifolium pratense. Total 20 kg/ha. Mixture: 27% HEBE  36% FIONA 36% MERULA (15 kg + 20 kg + 20 kg)"
df['Management_specific'] = df['Management_specific'].str.replace(s, 'HEBE-FIONA-MERULA')

s = "Type OH (HEBE FIONA TEDI). Trifolium pratense 4n Sorte TEDI - Trifolium repens Sorte HEBE - Trifolium repens Sorte FIONA. Total 200 g/a of each 1/3"
df['Management_specific'] = df['Management_specific'].str.replace(s, 'HEBE-FIONA-TEDI')

df['Management_specific'] = df['Management_specific'].str.replace('Hobelsaat (tillage until 3-4 cm)', '3-4CM')
df['Management_specific'] = df['Management_specific'].str.replace('XXX', 'XXX')
df['Management_specific'] = df['Management_specific'].str.replace('XXX', 'XXX')


def create_varname(row): 
    return "_".join([
        row['PREFIX'],        
        row['Management_Category'], 
        row['Management_Subcategory'],
        str(row['Management_specific']),
        row['Parcel']
    ])

df['VARNAME'] = df.apply(create_varname, axis=1)
df['VARNAME'] = df['VARNAME'].str.replace('_-9999', '')

# df['Management_Category'].unique()
# print(df['Management_Subcategory'].unique())
# print(df['Management_specific'].unique())
[print(f"{ix}: {v}") for ix, v in enumerate(df['VARNAME'].unique())];
# display(df)

0: MGMT_FERT_ORG_MAN_PARCEL-A
1: MGMT_PPGM_HERBREMOVAL_RUMEX_PARCEL-A
2: MGMT_PPGM_HARROWING_1CM_PARCEL-A
3: MGMT_USE_GRAZING_CATTLE_PARCEL-A
4: MGMT_PPGM_ROLLING_PARCEL-A
5: MGMT_PPGM_HARROWING_7CM_PARCEL-A
6: MGMT_SOWING_OVERSOWING_OH440RENO_PARCEL-A
7: MGMT_USE_MOWING_GRASSSILAGE_PARCEL-A
8: MGMT_FERT_MIN_CAN_PARCEL-A
9: MGMT_FERT_ORG_MAN_PARCEL-B
10: MGMT_FERT_ORG_SLU_PARCEL-B
11: MGMT_PPGM_HARROWING_1CM_PARCEL-B
12: MGMT_USE_MOWING_GRASSSILAGE_PARCEL-B
13: MGMT_PPGM_HERBREMOVAL_RUMEX_PARCEL-B
14: MGMT_USE_GRAZING_SHEEP_PARCEL-B
15: MGMT_FERT_ORG_SLU_PARCEL-A
16: MGMT_USE_GRAZING_SHEEP_PARCEL-A
17: MGMT_SOWING_OVERSOWING_OH440EXTRA_PARCEL-B
18: MGMT_USE_MOWING_AFTERGRASS_PARCEL-B
19: MGMT_FERT_MIN_CAN_PARCEL-B
20: MGMT_USE_MOWING_AFTERGRASS_PARCEL-A
21: MGMT_SOWING_OVERSOWING_OH440RENO_PARCEL-B
22: MGMT_USE_GRAZING_CATTLE_PARCEL-B
23: MGMT_PPGM_HERBICIDE_ASULOX_PARCEL-B
24: MGMT_PPGM_ROLLING_PARCEL-B
25: MGMT_SOILCULTIVATION_ROLLING_PARCEL-B
26: MGMT_USE_MOWING_HAY_PARCEL-A
27: MGM

In [50]:
mgmt = pd.DataFrame(index=maindf.index, columns=[])
mgmt

""
TIMESTAMP_MIDDLE
2005-01-01 00:15:00
2005-01-01 00:45:00
2005-01-01 01:15:00
2005-01-01 01:45:00
2005-01-01 02:15:00
...
2023-12-31 21:45:00
2023-12-31 22:15:00
2023-12-31 22:45:00


In [51]:
categories = df['Management_Category'].unique()
[print(ix, c) for ix, c in enumerate(categories)];

0 Fertilization
1 Plant protection and growth maintenance
2 Use
3 Sowing
4 Soil cultivation


In [52]:
subcategories = []
for ix, c in enumerate(categories):
    if ix > 0:
        break
    locs = df['Management_Category'] == c    
    subset = df.loc[locs, ['Parcel', 'Management_Category', 'Management_Subcategory', 'Management_specific', 'Start', 'End']]
    # display(subset)
    def test(row):
        # print(row['Parcel'])

        varname = 

        
        print(row['Start'])
    subset.apply(test, axis=1)
    

    # _subcategories = df.loc[locs, 'Management_Subcategory'].unique()    
    # for s in _subcategories:
    #     subcategories.append(_subcategories)    
    # print(subcategories)
    # print(f"{df.loc[locs, 'Management_Subcategory']}")

# m = df.loc[locs, ['Parcel', 'Management_Category', 'Management_Subcategory', 'Management_specific', 'Start', 'End']]
# df.loc[locs, :]
# m

2001-01-15 00:00:00
2001-08-06 00:00:00
2001-01-16 00:00:00
2001-03-30 00:00:00
2001-05-11 00:00:00
2002-03-05 00:00:00
2002-03-08 00:00:00
2002-04-12 00:00:00
2002-06-20 00:00:00
2002-08-20 00:00:00
2002-10-04 00:00:00
2002-11-21 00:00:00
2002-03-08 00:00:00
2002-06-20 00:00:00
2002-07-25 00:00:00
2002-10-04 00:00:00
2003-03-12 00:00:00
2003-08-05 00:00:00
2003-11-06 00:00:00
2003-03-12 00:00:00
2003-05-09 00:00:00
2003-08-07 00:00:00
2003-11-24 00:00:00
2004-02-16 00:00:00
2004-10-13 00:00:00
2004-11-08 00:00:00
2004-02-20 00:00:00
2004-05-18 00:00:00
2004-07-06 00:00:00
2004-08-04 00:00:00
2004-10-13 00:00:00
2004-11-08 00:00:00
2005-05-10 00:00:00
2005-06-02 00:00:00
2005-08-03 00:00:00
2005-12-05 00:00:00
2005-05-04 00:00:00
2005-06-02 00:00:00
2005-08-03 00:00:00
2005-12-05 00:00:00
2006-03-24 00:00:00
2006-05-05 00:00:00
2006-06-13 00:00:00
2006-07-06 00:00:00
2006-08-02 00:00:00
2006-10-19 00:00:00
2006-12-11 00:00:00
2006-03-24 00:00:00
2006-05-05 00:00:00
2006-06-13 00:00:00


In [37]:
# pd.to_datetime(m['Start'])